### Notebook poświęcony podziale danych na training set i test set dla metody collaborative filtering

#### Przygotowania

Import wymaganych bibliotek

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

Wczytywanie danych

In [32]:
ratings = pd.read_csv('../data/ratings.csv')
print("Ilość ocen w zbiorze: ", ratings.shape[0])
print("Przykładowe oceny:")
print(ratings.head(2))

Ilość ocen w zbiorze:  32000204
Przykładowe oceny:
   userId  movieId  rating  timestamp
0       1       17     4.0  944249077
1       1       25     1.0  944250228


#### Podział danych

Dane są dzielone zgodnie z następującymi zasadami:
- 20% danych jest przydzielanych do zbioru testowego
- 80% danych jest przydzielanych do zbioru treningowego
- podział ten jest wykonywany na ocenach każdego użytkownika (stratyfikacja po użytkownikach gwarantuje, że każdy użytkownik ma reprezentację w obu zbiorach)

In [33]:
train_data, test_data = train_test_split(
    ratings,
    test_size=0.2,
    stratify=ratings["userId"],
    random_state=264,
)

Tak jak widać w powyższym kodzie ustawione jest ziarno na pewną stałą wartość, aby podział danych zawsze przebiegał w ten sam sposób.
<br> Poniżej zostaną sprawdzone założenie podziału danych.

In [34]:
print(f"\nTrain: {len(train_data)} | Test: {len(test_data)}")
print(f"Odsetek użytkowników w zbiorze testowym: {test_data['userId'].nunique()/ratings['userId'].nunique():.1%}")


Train: 25600163 | Test: 6400041
Odsetek użytkowników w zbiorze testowym: 100.0%


Collaborative filtering nie będzie w stanie poprawnie wykonać predykcji dla filmów, na których nie został wytrenowany, dlatego w naszym zbiorze testowym należy wybrac tylko te filmy, które są w zbiorze treningowym.

In [35]:
train_movies = set(train_data["movieId"].unique())
test_data_clean = test_data[test_data["movieId"].isin(train_movies)]

Następnie te usunięte filmy ze zbioru testowego przeniesione zostaną do zbioru treningowego.

In [36]:
missing_in_test = test_data[~test_data["movieId"].isin(train_movies)]
final_train = pd.concat([train_data, missing_in_test])

Poniżej znajdują się wyniki zmian w zbiorach.

In [37]:
print(f"\nPo korekcie:")
print(f"Final Train: {len(final_train)}")
print(f"Test Clean: {len(test_data_clean)}")
print(f"Usuniętych z testu: {len(test_data)-len(test_data_clean)}")
print(f"Odsetek usiuniętych: {(len(test_data)-len(test_data_clean))/len(test_data):.3%}")


Po korekcie:
Final Train: 25604902
Test Clean: 6395302
Usuniętych z testu: 4739
Odsetek usiuniętych: 0.074%


Następnie zostanie zweryfikowany nasz podział.

1. Czy wszystkie filmy będące w zbiorze testowym są również w zbiorze treningowym?

In [38]:
assert test_data_clean["movieId"].nunique() == test_data_clean["movieId"].nunique(), "Niepoprawna liczba filmów w zbiorze testowym"

2. Czy wszyscy użytkownicy są obecni w obu zbiorach?

In [39]:
print(f"\nUżytkownicy w zbiorze treningowym: {final_train['userId'].nunique()}")
print(f"Użytkownicy w zbiorze testowym:    {test_data_clean['userId'].nunique()}")


Użytkownicy w zbiorze treningowym: 200948
Użytkownicy w zbiorze testowym:    200948


3. Czy oceny pojedyńczym użytkowniku podzieliły się w stosunku $1/5$?

In [40]:
sample_user1 = ratings['userId'].iloc[0]
sample_user2 = ratings['userId'].iloc[1000]
print(f"\nPierwszy przykładowy użytkownik: {sample_user1}")
print(f"Trening: {len(final_train[final_train['userId'] == sample_user1])} ocen")
print(f"Test:    {len(test_data_clean[test_data_clean['userId'] == sample_user1])} ocen")
print(f"\nDrugi przykładowy użytkownik: {sample_user2}")
print(f"Trening: {len(final_train[final_train['userId'] == sample_user2])} ocen")
print(f"Test:    {len(test_data_clean[test_data_clean['userId'] == sample_user2])} ocen")


Pierwszy przykładowy użytkownik: 1
Trening: 113 ocen
Test:    28 ocen

Drugi przykładowy użytkownik: 10
Trening: 528 ocen
Test:    132 ocen


4. Rzadkość macierzy w zbiorach

In [41]:
print(f"\nSparsity Trening: {(1 - len(final_train)/(final_train['userId'].nunique() * final_train['movieId'].nunique())):.2%}")
print(f"Sparsity Test: {(1 - len(test_data_clean)/(test_data_clean['userId'].nunique() * test_data_clean['movieId'].nunique())):.2%}")


Sparsity Trening: 99.85%
Sparsity Test: 99.94%


Zapis danych

In [42]:
final_train.to_csv('../data/cf/train_rating.csv', index=False)
test_data_clean.to_csv('../data/cf/test_rating.csv', index=False)